In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import datetime
import collections
# spark = SparkSession\
#     .builder\
#     .appName("MergeDataset")\
#     .getOrCreate()

c:\Users\super\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
player_attr = pd.read_csv('./data/player_attr.csv')
fbref_MatchPlayerStats = pd.read_csv('./data/fbref_MatchPlayerStats.csv')
fbref_MatchInfos = pd.read_csv('./data/fbref_MatchInfos.csv')

In [3]:
player_attr[player_attr.columns.values.tolist()] = player_attr[player_attr.columns.values.tolist()].astype(str)
fbref_MatchPlayerStats[fbref_MatchPlayerStats.columns.values.tolist()] = fbref_MatchPlayerStats[fbref_MatchPlayerStats.columns.values.tolist()].astype(str)
fbref_MatchInfos[fbref_MatchInfos.columns.values.tolist()] = fbref_MatchInfos[fbref_MatchInfos.columns.values.tolist()].astype(str)


fbref_MatchInfos['match_date'] = fbref_MatchInfos['match_date'].astype('datetime64[ns]')
fbref_MatchPlayerStats

,Ast,Blocks,Carries,Carries_PrgC,CrdR,CrdY,GCA,Gls,Int,PK,...,fbrefMatchId,minutes,nationality,npxG,player_kitnum,player_name,position,team,xAG,xG
0,0.0,1.0,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,db4d7668,90.0,ENG,0.8,9.0,Troy Deeney,FW,Watford,0.1,0.8
1,0.0,0.0,19.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,...,db4d7668,58.0,ESP,0.9,7.0,Gerard Deulofeu,FW,Watford,0.0,0.9
2,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,db4d7668,32.0,JAM,0.1,18.0,Andre Gray,FW,Watford,0.0,0.1
3,0.0,1.0,33.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,db4d7668,90.0,ARG,0.1,37.0,Roberto Pereyra,"AM,LM",Watford,0.1,0.1
4,0.0,0.0,32.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,db4d7668,66.0,ENG,0.2,19.0,Will Hughes,"AM,RM",Watford,0.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56029,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,e62f6e78,8.0,SCO,0.0,3.0,Kieran Tierney,LB,Arsenal,0.0,0.0
56030,0.0,2.0,48.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,e62f6e78,90.0,BRA,0.0,6.0,Gabriel Dos Santos,CB,Arsenal,0.0,0.0
56031,0.0,2.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,e62f6e78,90.0,FRA,0.0,12.0,William Saliba,CB,Arsenal,0.0,0.0
56032,0.0,4.0,24.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,...,e62f6e78,90.0,ENG,0.0,4.0,Ben White,RB,Arsenal,0.1,0.0


In [4]:
list_names = ['Troy','Deeney']

len([all(name in player_name for name in list_names) for player_name in player_attr['player_full_name'].tolist()])
len(player_attr)

85427

In [5]:
# This function merges the data from fbref, SoFifa and player attributes to create a merged dataset.
# It takes three parameters: fbref_MatchInfos, fbref_MatchPlayerStats and player_attr.
# It creates a dataframe called MergedSoFifaFbref and merges the data from fbref, SoFifa and player attributes.
# It then exports the merged dataset to a csv file called MergedSoFifaFbref.csv.
def MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr):
    # Create a dataframe called fbref_players which contains the player's kit number, name, position, team, nationality, fbrefMatchId and age from fbref_MatchPlayerStats.
    fbref_players = fbref_MatchPlayerStats[['player_kitnum','player_name','position','team','nationality','fbrefMatchId','age']]
    # Merge the dataframe fbref_players with fbref_MatchInfos on the fbrefMatchId.
    fbref_players = fbref_players.merge(fbref_MatchInfos[['match_date','fbrefMatchId']], on='fbrefMatchId', how = 'inner')
    # Convert the player's name, match_date, age and update_date to strings.
    fbref_players['player_name'] = fbref_players['player_name'].astype('str')
    fbref_players['match_date'] = fbref_players['match_date'].astype('datetime64[ns]')
    player_attr['update_date'] = player_attr['update_date'].astype('datetime64[ns]')
    fbref_players['age'] = fbref_players['age'].astype('str')
    # Create a list of the columns of the merged dataset.
    columns_MergedSoFifaFbref = fbref_players.columns.values.tolist()
    columns_MergedSoFifaFbref.extend(player_attr.columns.values.tolist())
    # Remove any duplicate columns from the list.
    columns_MergedSoFifaFbref = np.unique(columns_MergedSoFifaFbref).tolist()
    # Create a dataframe called MergedSoFifaFbref with the columns from the list.
    MergedSoFifaFbref = pd.DataFrame(columns=columns_MergedSoFifaFbref)
    # Iterate through each row of fbref_players.
    for i in range(0,len(fbref_players)):
    # for i in range(0,4):
        # Create a dataframe called fbref_player which contains the current row of fbref_players.
        fbref_player = fbref_players.loc[i]
        # Split the player's name into separate words.
        fbref_player_name = fbref_player['player_name']
        fbref_player_name = fbref_player_name.split(' ')
        # Get variables from the current row of fbref_players.
        match_date = fbref_player['match_date']
        player_kitnum = fbref_player['player_kitnum']
        team = fbref_player['team']
        age = fbref_player['age']
        # Take the first two characters from the age variable.
        age = age[0:2]
        
        # Filter Rơws in player_attr that have player_attr.player_full_name == fbref_player_name
        rows_contain_name = [all(name in player_full_name for name in fbref_player_name) for player_full_name in player_attr['player_full_name'].tolist()]
        df_sofifa = player_attr[rows_contain_name]
        # Filter the dataframe df_sofifa to only contain the current month and year of the match_date.
        df_sofifa = df_sofifa[(df_sofifa['update_date'].dt.month == match_date.month) & (df_sofifa['update_date'].dt.year == match_date.year)]
        # df_sofifa = df_sofifa[df_sofifa['team1_kitnum'] == player_kitnum]
        # Filter the dataframe df_sofifa to only contain the player's kit number and team.
        # df_sofifa = df_sofifa[df_sofifa['player_kitnum'] == player_kitnum]
        df_sofifa = df_sofifa[(df_sofifa['team1'] == team) | (df_sofifa['team2'] == team)]
        # Try to convert the age variable to an integer.
        # try:
        #     age = int(age)
        #     df_sofifa = df_sofifa[df_sofifa['age'] == age]
        # # If the conversion fails, pass.
        # except:
        #     pass
        # If the dataframe df_sofifa is not empty, create a dataframe called MergedSoFifaFbref_row which contains the current row of df_sofifa and the current row of fbref_players.
        if (len(df_sofifa)>0):
            MergedSoFifaFbref_row = df_sofifa.iloc[0:1]
            # Create a list of the columns of fbref_players and df_sofifa.
            delete_columns = fbref_players.columns.values.tolist()
            delete_columns.extend(MergedSoFifaFbref_row.columns.values.tolist())
            # Remove any duplicate columns from the list.
            delete_columns = [item for item, count in collections.Counter(delete_columns).items() if count > 1]
            MergedSoFifaFbref_row = MergedSoFifaFbref_row.drop(columns = delete_columns)
            # Reset the index of MergedSoFifaFbref_row.
            MergedSoFifaFbref_row.reset_index(drop=True, inplace=True)
            # Concatenate the current row of MergedSoFifaFbref_row and the current row of fbref_players.
            MergedSoFifaFbref_row = pd.concat([MergedSoFifaFbref_row.reset_index(drop=True),fbref_players.iloc[i:i+1].reset_index(drop=True)],axis=1)
        # If the dataframe df_sofifa is empty, create a dataframe called MergedSoFifaFbref_row which contains the current row of fbref_players.
        else:
            MergedSoFifaFbref_row = fbref_players.iloc[i:i+1]
        # Reset the index of MergedSoFifaFbref.
        MergedSoFifaFbref.reset_index(inplace=True, drop=True)
        # Concatenate the current row of MergedSoFifaFbref_row and MergedSoFifaFbref.
        MergedSoFifaFbref = pd.concat([MergedSoFifaFbref,MergedSoFifaFbref_row])
        
    # Export the merged dataset to a csv file called MergedSoFifaFbref.csv.
    MergedSoFifaFbref.to_csv('./data/MergedSoFifaFbref.csv',index=False,header=True,encoding='utf-8-sig')

MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr)

C:\Users\super\AppData\Local\Temp\ipykernel_12724\2651757376.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  MergedSoFifaFbref = pd.concat([MergedSoFifaFbref,MergedSoFifaFbref_row])


In [ ]:
# def MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr):
#     fbref_players = fbref_MatchPlayerStats[['player_kitnum','player_name','position','team','nationality','fbrefMatchId','age']]
#     fbref_players = fbref_players.merge(fbref_MatchInfos[['match_date','fbrefMatchId']], on='fbrefMatchId', how = 'inner')
#     fbref_players['player_name'] = fbref_players['player_name'].astype('str')
#     fbref_players['match_date'] = fbref_players['match_date'].astype('datetime64[ns]')
#     player_attr['update_date'] = player_attr['update_date'].astype('datetime64[ns]')
#     fbref_players['age'] = fbref_players['age'].astype('str')
    

# MergedSoFifaFbref(fbref_MatchInfos,fbref_MatchPlayerStats,player_attr)

In [69]:
fbref_MatchInfos['match_date'] = fbref_MatchInfos['match_date'].astype('datetime64[ns]')
fbref_MatchInfos['match_date'][1]

Timestamp('2020-07-26 00:00:00')

In [63]:

# names = ['Troy','Deeney','askjd']
# all([name in ['Troy Deeney'] for name in names])
print(fbref_MatchInfos.columns.values.tolist())


['attendance', 'away_team', 'fbrefMatchId', 'fbrefURL', 'home_team', 'leauge', 'match_date', 'match_date_str', 'match_week', 'officials', 'season', 'venue', 'venue_time']
